## We will collect spot data from https://data.binance.vision/?prefix=data/spot/daily/klines/BTCUSDT/1s/ :

In [1]:
import requests, zipfile, io, pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
from pathlib import Path

INPUTS_DIR = Path.cwd().parent / "data" / "inputs"
OUTPUT_FILE = INPUTS_DIR / "spot_full.csv"
START_DATE, END_DATE = "2023-05-18", "2023-09-07"
BASE_URL = "https://data.binance.vision/data/spot/daily/klines/BTCUSDT/1s/"

def download_and_process_spot():
    start, end = datetime.strptime(START_DATE, "%Y-%m-%d"), datetime.strptime(END_DATE, "%Y-%m-%d")
    delta = end - start
    dfs = []
    for i in tqdm(range(delta.days + 1)):
        url = BASE_URL + f"BTCUSDT-1s-{(start + timedelta(days=i)).strftime("%Y-%m-%d")}.zip"
        response = requests.get(url)
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            with z.open(z.namelist()[0]) as f:
                df = pd.read_csv(f, header=None, usecols=[0, 4])
                df.columns = ['timestamp', 'underlying_price']
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                dfs.append(df)

    full_df = pd.concat(dfs, ignore_index=True)
    full_df.to_csv(OUTPUT_FILE, index=False)

download_and_process_spot()

100%|██████████| 113/113 [04:24<00:00,  2.34s/it]
